In [13]:
from matrix_generator import generate_matrix
import subprocess

In [16]:
matrix_sizes = [0x200, 0x400, 0x600, 0x800, 0x1000, 0x1200, 0x1400, 0x1600, 0x1800, 0x2000]
patterns = ["checkerboard", "blockdiagonal", "random"]

In [17]:
#define test matrix sizes

matrix_files = []

# generate matrix input sizes
for matrix_size in matrix_sizes:
    for pattern in patterns:
        matrix_name = f"tests/MatrixA_{matrix_size}_{pattern}.mat"
        matrix_files.append((matrix_name, matrix_size, pattern))
        generate_matrix(output=matrix_name,
                        num_cols=matrix_size,
                        num_rows=matrix_size,
                        sparsity=0.0,
                        dtype='float16',
                        heatmap=False,
                        pattern=pattern,
                        blocksize=16)
        print("Created " + matrix_name)

Created tests/MatrixA_512_checkerboard.mat
Created tests/MatrixA_512_blockdiagonal.mat
Created tests/MatrixA_512_random.mat
Created tests/MatrixA_1024_checkerboard.mat
Created tests/MatrixA_1024_blockdiagonal.mat
Created tests/MatrixA_1024_random.mat
Created tests/MatrixA_1536_checkerboard.mat
Created tests/MatrixA_1536_blockdiagonal.mat
Created tests/MatrixA_1536_random.mat
Created tests/MatrixA_2048_checkerboard.mat
Created tests/MatrixA_2048_blockdiagonal.mat
Created tests/MatrixA_2048_random.mat
Created tests/MatrixA_4096_checkerboard.mat
Created tests/MatrixA_4096_blockdiagonal.mat
Created tests/MatrixA_4096_random.mat
Created tests/MatrixA_4608_checkerboard.mat
Created tests/MatrixA_4608_blockdiagonal.mat
Created tests/MatrixA_4608_random.mat
Created tests/MatrixA_5120_checkerboard.mat
Created tests/MatrixA_5120_blockdiagonal.mat
Created tests/MatrixA_5120_random.mat
Created tests/MatrixA_5632_checkerboard.mat
Created tests/MatrixA_5632_blockdiagonal.mat
Created tests/MatrixA_563

In [ ]:
# test the code
for i in range(0, len(matrix_files), 3):
    for run in range(30):
        print(f"Run {run}")
        print("Running " + matrix_files[i][0])
        result = subprocess.run(["./cmake-build-debug/matrix_multiplication",
                                 matrix_files[i][0], matrix_files[i+2][0]], stdout=subprocess.PIPE)
        print("Done running " + matrix_files[i][0])
        with open(f"tests/MatrixC_{matrix_files[i][1]}_{matrix_files[i][2]}_run{run}.res", "w") as f:
            f.write(result.stdout.decode('utf-8'))

        print("Running " + matrix_files[i+1][0])
        result = subprocess.run(["./cmake-build-debug/matrix_multiplication",
                                 matrix_files[i+1][0], matrix_files[i+2][0]], stdout=subprocess.PIPE)
        print("Done running " + matrix_files[i+1][0])
        with (open(f"tests/MatrixC_{matrix_files[i+1][1]}_{matrix_files[i+1][2]}_run{run}.res", "w")
              as f):
            f.write(result.stdout.decode('utf-8'))

Run 0
Running tests/MatrixA_512_checkerboard.mat
Done running tests/MatrixA_512_checkerboard.mat
Running tests/MatrixA_512_blockdiagonal.mat
Done running tests/MatrixA_512_blockdiagonal.mat
Run 1
Running tests/MatrixA_512_checkerboard.mat
Done running tests/MatrixA_512_checkerboard.mat
Running tests/MatrixA_512_blockdiagonal.mat
Done running tests/MatrixA_512_blockdiagonal.mat
Run 2
Running tests/MatrixA_512_checkerboard.mat
Done running tests/MatrixA_512_checkerboard.mat
Running tests/MatrixA_512_blockdiagonal.mat
Done running tests/MatrixA_512_blockdiagonal.mat
Run 3
Running tests/MatrixA_512_checkerboard.mat
Done running tests/MatrixA_512_checkerboard.mat
Running tests/MatrixA_512_blockdiagonal.mat
Done running tests/MatrixA_512_blockdiagonal.mat
Run 4
Running tests/MatrixA_512_checkerboard.mat
Done running tests/MatrixA_512_checkerboard.mat
Running tests/MatrixA_512_blockdiagonal.mat
Done running tests/MatrixA_512_blockdiagonal.mat
Run 5
Running tests/MatrixA_512_checkerboard.mat
D